In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
data = pickle.load(open('data_face_features.pickle',mode='rb'))

In [3]:
x = np.array(data['data'])
y = np.array(data['label'])

In [4]:
x.shape , y.shape

((40, 1, 128), (40,))

In [5]:
x = x.reshape(-1, 128)

In [6]:
x.shape

(40, 128)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.8, random_state=0)

In [9]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((32, 128), (8, 128), (32,), (8,))

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [11]:
# logistic regression

model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train)

LogisticRegression()

In [12]:
def get_report(model, x_train, y_train, x_test, y_test):
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)

    # accuracy_score
    acc_train = accuracy_score(y_train, y_pred_train)
    acc_test = accuracy_score(y_test, y_pred_test)

    # f1_score
    f1_score_train = f1_score(y_train,y_pred_train, average='macro')
    f1_score_test = f1_score(y_test, y_pred_test, average='macro')
    
    print('Accuracy Train: ', acc_train)
    print('Accuracy Test: ', acc_test)
    print('f1_score Train: ', f1_score_train)
    print('f1_score Test: ', f1_score_test)

In [13]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accuracy Train:  0.96875
Accuracy Test:  0.375
f1_score Train:  0.9641148325358851
f1_score Test:  0.3333333333333333


In [14]:
# Support Vector Machine

In [15]:
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)
get_report(model_svc, x_train, y_train, x_test, y_test)

Accuracy Train:  0.96875
Accuracy Test:  0.5
f1_score Train:  0.9641148325358851
f1_score Test:  0.3916666666666666


In [16]:
# random forest

In [17]:
model_rf = RandomForestClassifier(n_estimators=10)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=10)

In [18]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accuracy Train:  1.0
Accuracy Test:  0.375
f1_score Train:  1.0
f1_score Test:  0.35


In [19]:
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier())
],voting='soft',weights=[2,3,1])

In [20]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft', weights=[2, 3, 1])

In [21]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accuracy Train:  0.96875
Accuracy Test:  0.5
f1_score Train:  0.9641148325358851
f1_score Test:  0.3916666666666666


In [22]:
from sklearn.model_selection import GridSearchCV

In [23]:
model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1,0.3,0.5],
    'rf__n_estimators': [5,10,20],
    'rf__max_depth': [3,5,7],
    'voting': ['soft','hard']
}, scoring='accuracy', cv=3,n_jobs=1,verbose=2)

In [24]:
model_grid.fit(x_train,y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft 
[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft 
[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft 
[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard 
[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard 
[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=3, 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=hard, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=hard 
[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=hard, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=hard 
[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=hard, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft 
[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft 
[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft 
[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft, total=   0.0s
[CV] rf__max_depth=

[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=10, svm__gamma=0.3, voting=hard 
[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=10, svm__gamma=0.3, voting=hard, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=10, svm__gamma=0.3, voting=hard 
[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=10, svm__gamma=0.3, voting=hard, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=10, svm__gamma=0.3, voting=hard 
[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=10, svm__gamma=0.3, voting=hard, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=10, svm__gamma=0.5, voting=soft 
[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=10, svm__gamma=0.5, voting=soft, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=10, svm__gamma=0.5, voting=soft 
[CV]  rf__max_depth=3, rf__n_estimators=5, svm__C=10, svm__gamma=0.5, voting=soft, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=5, svm__C=10, svm__gamma=0.5, voting=soft 
[CV]  rf__max_depth=3, 

[CV]  rf__max_depth=3, rf__n_estimators=10, svm__C=7, svm__gamma=0.1, voting=hard, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=10, svm__C=7, svm__gamma=0.3, voting=soft 
[CV]  rf__max_depth=3, rf__n_estimators=10, svm__C=7, svm__gamma=0.3, voting=soft, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=10, svm__C=7, svm__gamma=0.3, voting=soft 
[CV]  rf__max_depth=3, rf__n_estimators=10, svm__C=7, svm__gamma=0.3, voting=soft, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=10, svm__C=7, svm__gamma=0.3, voting=soft 
[CV]  rf__max_depth=3, rf__n_estimators=10, svm__C=7, svm__gamma=0.3, voting=soft, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=10, svm__C=7, svm__gamma=0.3, voting=hard 
[CV]  rf__max_depth=3, rf__n_estimators=10, svm__C=7, svm__gamma=0.3, voting=hard, total=   0.0s
[CV] rf__max_depth=3, rf__n_estimators=10, svm__C=7, svm__gamma=0.3, voting=hard 
[CV]  rf__max_depth=3, rf__n_estimators=10, svm__C=7, svm__gamma=0.3, voting=hard, total=   0.0s
[CV] rf_

...

[CV]  rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard, total=   0.0s
[CV] rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft 
[CV]  rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft, total=   0.0s
[CV] rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft 
[CV]  rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft, total=   0.0s
[CV] rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft 
[CV]  rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft, total=   0.0s
[CV] rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=hard 
[CV]  rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=hard, total=   0.0s
[CV] rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=hard 
[CV]  rf__max_depth=7, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=hard, total=   0.0s
[CV] rf_

[CV]  rf__max_depth=7, rf__n_estimators=10, svm__C=10, svm__gamma=0.3, voting=soft, total=   0.0s
[CV] rf__max_depth=7, rf__n_estimators=10, svm__C=10, svm__gamma=0.3, voting=hard 
[CV]  rf__max_depth=7, rf__n_estimators=10, svm__C=10, svm__gamma=0.3, voting=hard, total=   0.0s
[CV] rf__max_depth=7, rf__n_estimators=10, svm__C=10, svm__gamma=0.3, voting=hard 
[CV]  rf__max_depth=7, rf__n_estimators=10, svm__C=10, svm__gamma=0.3, voting=hard, total=   0.0s
[CV] rf__max_depth=7, rf__n_estimators=10, svm__C=10, svm__gamma=0.3, voting=hard 
[CV]  rf__max_depth=7, rf__n_estimators=10, svm__C=10, svm__gamma=0.3, voting=hard, total=   0.0s
[CV] rf__max_depth=7, rf__n_estimators=10, svm__C=10, svm__gamma=0.5, voting=soft 
[CV]  rf__max_depth=7, rf__n_estimators=10, svm__C=10, svm__gamma=0.5, voting=soft, total=   0.0s
[CV] rf__max_depth=7, rf__n_estimators=10, svm__C=10, svm__gamma=0.5, voting=soft 
[CV]  rf__max_depth=7, rf__n_estimators=10, svm__C=10, svm__gamma=0.5, voting=soft, total=   0.

[CV]  rf__max_depth=7, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=soft, total=   0.0s
[CV] rf__max_depth=7, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=hard 
[CV]  rf__max_depth=7, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=hard, total=   0.0s
[CV] rf__max_depth=7, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=hard 
[CV]  rf__max_depth=7, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=hard, total=   0.0s
[CV] rf__max_depth=7, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=hard 
[CV]  rf__max_depth=7, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=hard, total=   0.0s
[CV] rf__max_depth=7, rf__n_estimators=20, svm__C=7, svm__gamma=0.3, voting=soft 
[CV]  rf__max_depth=7, rf__n_estimators=20, svm__C=7, svm__gamma=0.3, voting=soft, total=   0.0s
[CV] rf__max_depth=7, rf__n_estimators=20, svm__C=7, svm__gamma=0.3, voting=soft 
[CV]  rf__max_depth=7, rf__n_estimators=20, svm__C=7, svm__gamma=0.3, voting=soft, total=   0.0s
[CV] rf_

[Parallel(n_jobs=1)]: Done 648 out of 648 | elapsed:   20.2s finished


GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier())],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [25]:
model_best_estimator = model_grid.best_estimator_

In [26]:
model_grid.best_score_

0.903030303030303

In [27]:
pickle.dump(model_best_estimator, open('./models/ml_face_person_identity.pkl',mode='wb'))